In [1]:
import os
import logging
import faiss
import tensorflow as tf

import deepr as dpr

from deepr.examples import recogym



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.











In [2]:
logging.basicConfig(level=logging.INFO)

In [3]:
path_root = "wan"
path_data = path_root + "/data"

path_mapping = path_data + "/mapping.txt"
path_train = path_data + "/train.tfrecord.gz"
path_eval = path_data + "/eval.tfrecord.gz"
path_test = path_data + "/test.tfrecord.gz"

In [4]:
build = recogym.jobs.BuildRecords(
    path_train=path_train,
    path_eval=path_eval,
    path_test=path_test
)

In [5]:
build.run()

Organic Users: 0it [00:00, ?it/s]
Users: 100%|██████████| 10000/10000 [02:31<00:00, 66.19it/s]
INFO:deepr.examples.recogym.jobs.build_records:Writing 1005572 timelines to wan/data/train.tfrecord.gz
INFO:deepr.examples.recogym.jobs.build_records:shuffle_timelines = True, num_negatives = 8
INFO:deepr.writers.record:Writing tf record dataset to wan/data/train.tfrecord.gz



INFO:deepr.readers.base:Reached end of DatasetReader(dataset=<DatasetV1Adapter shapes: (), types: tf.string>)
INFO:deepr.utils.iter:Number of items 9919, item/sec = 282.56
INFO:deepr.examples.recogym.jobs.build_records:Writing 1177 timelines to wan/data/test.tfrecord.gz
INFO:deepr.examples.recogym.jobs.build_records:shuffle_timelines = True, num_negatives = 8
INFO:deepr.writers.record:Writing tf record dataset to wan/data/test.tfrecord.gz
INFO:deepr.readers.base:Reached end of DatasetReader(dataset=<DatasetV1Adapter shapes: (), types: tf.string>)
INFO:deepr.utils.iter:Number of items 10, item/sec = 103.62
INFO:deepr.examples.recogym.jobs.build_records:Writing 1177 timelines to wan/data/eval.tfrecord.gz
INFO:deepr.examples.recogym.jobs.build_records:shuffle_timelines = True, num_negatives = 8
INFO:deepr.writers.record:Writing tf record dataset to wan/data/eval.tfrecord.gz
INFO:deepr.readers.base:Reached end of DatasetReader(dataset=<DatasetV1Adapter shapes: (), types: tf.string>)
INFO:

In [6]:
path_model = path_root + "/model"
path_variables = path_root + "/variables"
path_saved_model = path_root + "/saved_model"
path_predictions = path_root + "/predictions.parquet.snappy"




max_steps = 100_000
P = 10

train = dpr.jobs.Trainer(
    path_model=path_model,
    pred_fn=recogym.layers.AverageModel(vocab_size=P, dim=100),
    loss_fn=recogym.layers.BPRLoss(vocab_size=10, dim=100),
    optimizer_fn=dpr.optimizers.TensorflowOptimizer("LazyAdam", 0.001),
    train_input_fn=dpr.readers.TFRecordReader(path_train),
    eval_input_fn=dpr.readers.TFRecordReader(path_eval, shuffle=False),
    prepro_fn=recogym.prepros.RecordPrepro(
        min_input_size=3,
        min_target_size=3,
        max_input_size=50,
        max_target_size=50,
        buffer_size=1024,
        batch_size=128,
        repeat_size=None,
        prefetch_size=1,
        num_parallel_calls=8,
    ),
    train_spec=dpr.jobs.TrainSpec(max_steps=max_steps),
    eval_spec=dpr.jobs.EvalSpec(steps=None, start_delay_secs=30, throttle_secs=30),
    final_spec=dpr.jobs.FinalSpec(steps=None),
    exporters=[
        # The training will keep the model with the best triplet precision
        dpr.exporters.BestCheckpoint(metric="triplet_precision", mode="increase"),
        # Export biases and embeddings as a dataframe
        dpr.exporters.SaveVariables(path_variables=path_variables, variable_names=["biases", "embeddings"]),
        # Export a saved model using specified fields as input
        dpr.exporters.SavedModel(
            path_saved_model=path_saved_model,
            fields=[
                dpr.Field(name="inputPositives", shape=(None,), dtype=tf.int64),
                dpr.Field(name="inputMask", shape=(None,), dtype=tf.bool),
            ],
        ),
    ],
    train_hooks=[
        # Log metrics, hyperparams, initial values to the console, and optionally mlflow and graphite
        dpr.hooks.LoggingTensorHookFactory(
            name="training",
            functions={
                "memory_gb": dpr.hooks.ResidentMemory(unit="gb"),
                "max_memory_gb": dpr.hooks.MaxResidentMemory(unit="gb"),
            },
            every_n_iter=300,
            use_graphite=False,
            use_mlflow=False,
        ),
        dpr.hooks.SummarySaverHookFactory(save_steps=300),
        dpr.hooks.NumParamsHook(use_mlflow=False),
        dpr.hooks.LogVariablesInitHook(use_mlflow=False),
        dpr.hooks.StepsPerSecHook(
            name="training",
            batch_size=128,
            every_n_steps=300,
            skip_after_step=max_steps,
            use_mlflow=False,
            use_graphite=False,
        ),
        # Stop the training if triplet precision does not improve
        dpr.hooks.EarlyStoppingHookFactory(
            metric="triplet_precision",
            mode="increase",
            max_steps_without_improvement=1000,
            min_steps=5_000,
            run_every_steps=300,
            final_step=max_steps,
        ),
    ],
    eval_hooks=[dpr.hooks.LoggingTensorHookFactory(name="validation", at_end=True)],
    final_hooks=[dpr.hooks.LoggingTensorHookFactory(name="final_validation", at_end=True)],
    train_metrics=[dpr.metrics.StepCounter(name="num_steps"), dpr.metrics.DecayMean(tensors=["loss"], decay=0.98)],
    eval_metrics=[dpr.metrics.Mean(tensors=["loss", "triplet_precision"])],
    final_metrics=[dpr.metrics.Mean(tensors=["loss", "triplet_precision"])],
    run_config=dpr.jobs.RunConfig(
        save_checkpoints_steps=300, save_summary_steps=300, keep_checkpoint_max=None, log_step_count_steps=300
    ),
    config_proto=dpr.jobs.ConfigProto(
        inter_op_parallelism_threads=8, intra_op_parallelism_threads=8, gpu_device_count=0, cpu_device_count=48,
    ),
)

In [7]:
predict = recogym.jobs.Predict(
    path_saved_model=path_saved_model,
    path_predictions=path_predictions,
    input_fn=dpr.readers.TFRecordReader(path_test, shuffle=False),
    prepro_fn=recogym.prepros.RecordPrepro(),
)

In [8]:
evaluate = [
    recogym.jobs.Evaluate(
        path_predictions=path_predictions,
        path_embeddings=path_variables + "/embeddings",
        path_biases=path_variables + "/biases",
        k=k,
    )
    for k in [10, 20, 50]
]


In [9]:
pipeline = dpr.jobs.Pipeline([train, predict] + evaluate)

In [10]:
pipeline.run()

INFO:tensorflow:Using config: {'_model_dir': 'wan/model/checkpoints', '_tf_random_seed': None, '_save_summary_steps': 300, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': device_count {
  key: "CPU"
  value: 48
}
device_count {
  key: "GPU"
  value: 0
}
intra_op_parallelism_threads: 8
inter_op_parallelism_threads: 8
, '_keep_checkpoint_max': None, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 300, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x15e1c5c88>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'wan/model/checkpoints', '_tf_random_seed': None, '_save_summary_steps': 300, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': device_count {
  key: "CPU"
  value: 48
}
device_count {
  key: "GPU"
  value: 0
}
intra_op_parallelism_threads: 8
inter_op_parallelism_threads: 8
, '_keep_checkpoint_max': None, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 300, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x15e1c5c88>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Not using Distribute Coordinator.


INFO:tensorflow:Not using Distribute Coordinator.


INFO:tensorflow:Running training and evaluation locally (non-distributed).


...

INFO:tensorflow:Restoring parameters from wan/saved_model/1597944408/variables/variables


INFO:tensorflow:Restoring parameters from wan/saved_model/1597944408/variables/variables



INFO:deepr.predictors.saved_model:Retrieving feeds from default signature def
INFO:deepr.predictors.saved_model:Retrieving fetches from default signature def
INFO:deepr.prepros.combinators:Not applying Filter(IsMinSize(1, 1, inputs='inputPositives', outputs='t_0', name=None)) (mode=infer)
INFO:deepr.prepros.combinators:Not applying Filter(IsMinSize(1, 1, inputs='targetPositives', outputs='t_0', name=None)) (mode=infer)
INFO:deepr.prepros.core:Not applying Shuffle(1024) (mode=infer)
INFO:deepr.prepros.core:Not applying Repeat(None) (mode=infer)


Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.


Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
INFO:deepr.io.parquet:Writing table to wan/predictions.parquet.snappy
INFO:deepr.examples.recogym.jobs.predict:Wrote predictions to wan/predictions.parquet.snappy
INFO:deepr.examples.recogym.jobs.evaluate:precision@10 = 0.1
recall@10 = 1.0
f1@10 = 0.18181818181818182
INFO:deepr.examples.recogym.jobs.evaluate:precision@20 = 0.09090909090909091
recall@20 = 1.0
f1@20 = 0.16666666666666669
INFO:deepr.examples.recogym.jobs.evaluate:precision@50 = 0.09090909090909091
recall@50 = 1.0
f1@50 = 0.16666666666666669
